## 1. ACCESSING DATA

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer



#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv('hr_raw_data_v1.csv')

In [3]:
df.head(1)


,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,terminated,year_at_comp,year_current_role,standard_hours,remote,business_travel,over_time,job_level,stock_opt_level,traning_times_last_year,perf_rate,year_last_promotion,year_current_mngr,education_field,education_scale,annual_salary,monthly_income,daily_rate,hourly_rate,monthly_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,0,1972,51,NaN,6,resEArch DIREcToR,NaN,No,20,NaN,Full Time,Yes,NaN,No,5,0,5,"3,0",15,15,NaN,3,"195370,00$","16280,83$",2015.722222,NaN,"42330,17$",13,1,3,3,3,"3,0",7,NaN


## 2. DATA CLEANING

In [4]:
# Values format to title and strip

to_title = ['marital_status', 'job_title',  'department', 'education_field']

df[to_title] = df[to_title].apply(lambda x: x.str.title().str.strip() if x.dtype == 'object' else x)

In [5]:
# Marital status typos fixing

df["marital_status"]=df["marital_status"].replace("Marreid", "Married")

In [6]:
# Converting obj to float

convert_float = ['annual_salary','monthly_income','daily_rate']
def replace_currency(value):
        try:
                value = float(value.replace("$", "").replace(",", "."))
                return value
        except:
            return value



for col in convert_float:
     df[col]=df[col].apply(replace_currency)

In [7]:
# Replacing values obj to numeric in age

age_update = {
    'thirty-two': 32,
    'twenty-four': 24,
    'thirty': 30,
    'fifty-eight': 58,
    'fifty-two': 52,
    'twenty-six': 26,
    'fifty-five': 55,
    'thirty-seven': 37,
    'thirty-six': 36,
    'forty-seven': 47,
    'thirty-one': 31}

df['age'] = df['age'].replace(age_update)

In [8]:
# Gender update, where 0 is male and 1 is female
gender_map = {0: "M", 1:"F"}
df['gender'] = df['gender'].map(gender_map)

df['gender'].unique()

array(['M', 'F'], dtype=object)

## 3. NULLS MANAGEMENT

In [9]:
# RELATED SALARY COLUMNS IMPUTATION

salary_base =['monthly_income', 'daily_rate', 'annual_salary']

imputer = KNNImputer(n_neighbors=3)
df[salary_base] = imputer.fit_transform(df[salary_base])

In [10]:
# CONFIRMATION: RELATED SALARY COLUMNS IMPUTATION 
df[salary_base].isnull().sum()

monthly_income    0
daily_rate        0
annual_salary     0
dtype: int64

In [11]:
df['remote'].unique()

array(['Yes', '1', 'False', '0', 'True'], dtype=object)

In [12]:
# REMOTE WORKER HOMOGENIZATION


remote_update = {'True':'Yes',
                 '1': 'Yes',
                'False': 'No',
                '0': 'No'}

df['remote'] = df['remote'].replace(remote_update )

In [13]:
departments_dict =  {"Research & Development": ["Healthcare Representative", "Laboratory Technician", "Manufacturing Director", "Research Scientist", "Research Director"],
                    "Sales": ["Sales Executive", "Sales Representative"],
                    "Human Resources": ["Human Resources"]}

df['department'] = df['department'].fillna(df['job_title'].map(departments_dict))



In [14]:
departments_dict = {
    "Research & Development": ["Healthcare Representative", "Laboratory Technician", "Manufacturing Director", "Research Scientist", "Research Director"],
    "Sales": ["Sales Executive", "Sales Representative"],
    "Human Resources": ["Human Resources"]
}

# Reverse the dictionary to map job titles to departments
job_to_department = {job: dept for dept, jobs in departments_dict.items() for job in jobs}


def map_department(job_title):
    try:
        return job_to_department[job_title]
    except:
        return None 

# Apply the function to fill missing department values
df['department'] = df['department'].fillna(df['job_title'].apply(map_department))



In [15]:
df['department'].isna().sum()

95

## EXPORTING NEW DATA

In [16]:
df.to_csv("hr_raw_data_v2.csv", index=False)
